# Merge Fire Data with Land Cover

In [ ]:
import geopandas as gpd
import pandas as pd

algeria_lc = gpd.read_file('..\\Land Cover dataset\\merged_DZA_TUN_cleaned.shp')
df_csv = pd.read_csv('..\\fire_test\\fire_alg_tun.csv')
gdf_points = gpd.GeoDataFrame(
    df_csv,
    geometry=gpd.points_from_xy(df_csv['longitude'], df_csv['latitude']),
    crs="EPSG:4326"
)
gdf_points = gdf_points.to_crs(algeria_lc.crs)
merged = gpd.sjoin_nearest(
    gdf_points,
    algeria_lc,
    how="inner",
    distance_col="dist_to_poly"
)

merged = merged.drop(columns=['geometry', 'index_right',"dist_to_poly"])
merged.to_csv('..\\fire_test\\viirs-jpss1_2024_enriched_approx.csv', index=False)




# Add Elevation Data

In [ ]:
import rasterio
df = pd.read_csv("..\\fire_test\\viirs-jpss1_2024_enriched_approx.csv")
src = rasterio.open("..\\elevation_algeria_tunisia_cleaned.tif")

# Convertir lon/lat en index raster
rows, cols = rasterio.transform.rowcol(
    src.transform,
    df["longitude"].values,
    df["latitude"].values
)

# Lire les valeurs
band = src.read(1)

df["elevation"] = band[rows, cols]
df.to_csv("..\\fire_test\\merged_fire_landcover_elevation.csv", index=False)